In [2]:
# Had numpy version error. Uninstalled it and installed it back.
# !pip uninstall numpy --yes

Uninstalling numpy-1.17.0:
  Successfully uninstalled numpy-1.17.0


In [3]:
# Version 1.17 is not compatible with some other packages
# !pip install "numpy<1.17"

    100% |████████████████████████████████| 13.9MB 890kB/s ta 0:00:01


In [4]:
import numpy as np
import pandas as pd
import re
import json
from selenium.webdriver import Chrome, Firefox
from collections import Counter
import urllib.request
from functions import get_posts, get_hashtags, get_image, get_full_info
import boto3
from io import BytesIO
from os import walk

### Hashtags to scrape: 
1. travel
2. food
3. animals
4. selfie
5. cars
6. fitness
7. babies
8. wedding
9. nature
10. architecture

### Scraping:
Hashtags (from comments), Image Link, Image Name, Post Link, and Input Hashtag.  Then exporting the data to .json files, stored in a seperate folder.

*get_full_info* function also retrieves and downloads the actual image, which are stored locally to optimize possible space constraints.

In [5]:
# hashtags = ['travel', 'food', 'animals']
hashtags = ['nature']
# hashtags = ['babies', 'wedding', 'nature', 'architecture']
# hashtags = ['wedding']

In [6]:
for hashtag in hashtags:
    hashtag_data = get_full_info(hashtag, 300)

    with open(f'json_files/{hashtag}.json', 'w') as f:
        json.dump(hashtag_data, f)

### Import JSON files back into pandas dataframe

In [ ]:
travel_df = pd.read_json('json_files/travel.json')
travel_df.head()

### Upload images from common folder to S3 bucket folders by hashtag (Anna)
Scraped images were all in one folder independently of the category. Had to separate them by hashtag

In [ ]:
#hashtags to separate
h_list = ['travel', 'food', 'animals']

#import json files
data = []
for h in h_list:    
    with open(f'json_files/{h}.json', 'r') as f:
        for line in f:
            data.append(json.loads(line))

In [ ]:
#combine json files for all 3 hashtags
data_full = data[0] + data[1] + data[2]
len(data_full)

In [ ]:
#access s3 with boto3 and upload images
s3 = boto3.resource('s3')

for h in h_list:
    for i in data_full:
        if i['search_hashtag'] == h:
            source = 'data/'+ i['image_local_name']
            bucket = 'instagram-images-mod4'
            destination = i['search_hashtag'] + '/' + i['image_local_name']
            s3.meta.client.upload_file(source, bucket, destination)